In [1]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
model_type = "whisper-large-v2"
model_name = f"openai/{model_type}"
num_val_samples = 1024 

In [4]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    model_name, language="bengali", task="transcribe"
)

In [5]:
# import pandas as pd
# import os
# from torch.utils.data import Dataset
# import librosa
# from datasets import Audio

# class AudioDataset(Dataset):
#     def __init__(self, labels_df: pd.DataFrame, data_path: str, processor):
#         self.labels_df = labels_df
#         self.data_path = data_path
#         self.processor = processor
#     def __len__(self):
#         return len(self.labels_df)
#     def __getitem__(self,idx):
#         row = self.labels_df.iloc[idx]
#         path = os.path.join(self.data_path, row["id"]+".mp3")
#         sentence = row["sentence"]
#         with open(path, "rb") as f:
#             speech = f.read()
#             audio = Audio(sampling_rate=processor.feature_extractor.sampling_rate).decode_example({"path": path, "bytes": speech})

#         example = processor(audio=audio["array"], sampling_rate=processor.feature_extractor.sampling_rate, text=sentence)
#         example["input_length"] = len(audio["array"]) // processor.feature_extractor.sampling_rate
#         # speech, sr = librosa.load(path, sr=processor.feature_extractor.sampling_rate) 
# #         print(speech.shape)
#         return example

In [6]:
# from datasets import DatasetDict
# labels_df = pd.read_csv("bengaliai-speech/train.csv")

In [7]:
# from sklearn.model_selection import train_test_split
# train_df, val_df = train_test_split(labels_df, test_size=0.1, random_state=42)

In [8]:
# common_voice = DatasetDict()
# train_dataset= AudioDataset(train_df, data_path="bengaliai-speech/train_mp3s", processor=processor)
# test_dataset = AudioDataset(val_df, data_path="bengaliai-speech/train_mp3s", processor=processor)

In [9]:
# common_voice = DatasetDict({"train":train_dataset,"test": test_dataset})

# Dataset
## Kaggle Dataset

In [10]:
from datasets import DatasetDict, Audio

bengali_dataset = DatasetDict.load_from_disk("/data/abdalla/bengaliai-speech_huggingface")

In [11]:
def prepare_dataset(batch):
    audio = batch["audio"]
    sentence = batch["sentence"]
    # batched output is "un-batched"
    example = processor(audio=audio["array"], sampling_rate=16_000, text=sentence)
    example["input_length"] = len(audio["array"]) // processor.feature_extractor.sampling_rate

    return example

In [12]:
bengali_dataset["train"] = bengali_dataset["train"].map(prepare_dataset, remove_columns=bengali_dataset["train"].column_names, num_proc=128)

In [13]:
bengali_dataset["validation"] = bengali_dataset["validation"].map(prepare_dataset, remove_columns=bengali_dataset["validation"].column_names, num_proc=32)

## Mozilla Common Voice Dataset

In [14]:
# from datasets import load_dataset, load_metric, Audio

# common_voice_train = load_dataset("mozilla-foundation/common_voice_13_0", "bn", split="train", use_auth_token=True)
# common_voice_val = load_dataset("mozilla-foundation/common_voice_13_0", "bn", split="test", use_auth_token=True)

In [15]:
# common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes", "path", "variant"])
# common_voice_val = common_voice_val.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes", "path", "variant"])

In [16]:
# common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
# common_voice_val = common_voice_val.cast_column("audio", Audio(sampling_rate=16_000))

In [17]:
# common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
# common_voice_val = common_voice_val.map(prepare_dataset, remove_columns=common_voice_val.column_names)

In [18]:
# from datasets import concatenate_datasets

# combined_train = concatenate_datasets([bengali_dataset["train"], common_voice_train])

In [19]:
# combined_val = concatenate_datasets([bengali_dataset["validation"], common_voice_val])

In [20]:
# dataset = DatasetDict({"train": combined_train, "validation": combined_val})
dataset = bengali_dataset

In [21]:
import pandas as pd 
dataset["validation"] = dataset["validation"].select(pd.Series(list(range(len(dataset["validation"])))).sample(num_val_samples, random_state=42).values)

In [22]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union



class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __init__(self, processor):
        self.processor = processor

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [23]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [24]:
import evaluate

metric = evaluate.load("wer")

In [25]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}


In [26]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name)

In [27]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language="bengali", task="transcribe", use_cache=True
)

In [28]:
import os
import tempfile
import shutil
import json
import kaggle
from pathlib import Path
from transformers import TrainerCallback
from transformers.trainer_callback import TrainerControl, TrainerState
from transformers.training_args import TrainingArguments

class KaggleUploader(TrainerCallback):
    def __init__(self, dataset_path: str, id: str, title: str, isPrivate: bool):
        self.api = kaggle.KaggleApi()
        self.api.authenticate()
        self.dataset_path = dataset_path
        self.meta_data = dict(
            id=id,
            title=title,
            isPrivate=isPrivate,
            licenses=[dict(name="other")]
        )
        self.previous_best = None

    def on_save(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        best_model_checkpoint = str(Path(state.best_model_checkpoint).name)
        if best_model_checkpoint != self.previous_best:
            print(best_model_checkpoint)
            self.upload_dataset_to_kaggle(self.dataset_path, best_model_checkpoint)
            self.previous_best = best_model_checkpoint
        
        return super().on_evaluate(args, state, control, **kwargs)
    
    def upload_dataset_to_kaggle(self, dataset_path, checkpoint_to_save: str):
        # latest_checkpoint = find_latest_checkpoint(dataset_path)
        checkpoint = os.path.join(dataset_path, checkpoint_to_save)

        version_notes = checkpoint_to_save
        # The checkpoint has multiple files that we don't need.
        # We only need the pytorch_model.bin file, config.json and generation_config.json
        # Copy these files to a temporary folder
        with tempfile.TemporaryDirectory() as temp_dir:
            # create a directory inside named "model"
            temp_model_dir = os.path.join(temp_dir, "model")
            os.mkdir(temp_model_dir)
            # copy the files
            for file in ["pytorch_model.bin", "config.json", "generation_config.json"]:
                shutil.copy(os.path.join(checkpoint, file), temp_model_dir)

            # create dataset-metadata.json inside the temporary directory
            with open(os.path.join(temp_model_dir, "dataset-metadata.json"), "w") as f:
                json.dump(self.meta_data, f)

            self.api.dataset_create_version(temp_model_dir, version_notes=version_notes, dir_mode="zip")

In [29]:
kaggle_uploader = KaggleUploader("/workspaces/HuggingFace-Audio-Course/whisper-large-v2-bn",
                                 id="kurokabe/whisper-large-bn",
                                 title="Whisper large bengali", 
                                 isPrivate=True)

In [37]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=f"./{model_type}-bn",  # name on the HF Hub
    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,  # increase by 2x for every 2x decrease in batch size
    group_by_length=True, 
    length_column_name="input_length",
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    # warmup_steps=50,
    num_train_epochs=10,
    # max_steps=100000,  # increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=250,
    eval_steps=250,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="eval_wer_ortho",
    greater_is_better=False,
    push_to_hub=True,
    save_total_limit=2,
    # dataloader_num_workers=64,
)


In [38]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    callbacks=[kaggle_uploader]
)

/workspaces/HuggingFace-Audio-Course/./whisper-large-v2-bn is already a clone of https://huggingface.co/Kurokabe/whisper-large-v2-bn. Make sure you pull the latest changes with `repo.git_pull()`.


In [39]:
while True:
    try:
        trainer.train(resume_from_checkpoint=True)
    except RuntimeError as e:
        print(e)
        continue

Step,Training Loss,Validation Loss,Wer Ortho,Wer
20250,0.184500,0.042081,0.197688,0.087373
20500,0.177900,0.041363,0.198006,0.088055


checkpoint-20250
Starting upload for file pytorch_model.bin


100%|██████████| 5.75G/5.75G [02:32<00:00, 40.6MB/s]


Upload successful: pytorch_model.bin (6GB)
Starting upload for file generation_config.json


100%|██████████| 4.25k/4.25k [00:01<00:00, 3.92kB/s]


Upload successful: generation_config.json (4KB)
Starting upload for file config.json


100%|██████████| 2.23k/2.23k [00:00<00:00, 3.06kB/s]


Upload successful: config.json (2KB)


In [33]:
kwargs = {
    "model_name": f"{model_type} bn",  # a 'pretty' name for your model
    "language": "bn",
    "finetuned_from": model_name,
    "tasks": "automatic-speech-recognition",
}
trainer.push_to_hub(**kwargs)

Upload file pytorch_model.bin:   0%|          | 1.00/5.75G [00:00<?, ?B/s]

Upload file runs/Aug20_08-27-34_77f3c72081bc/events.out.tfevents.1692520070.77f3c72081bc:   0%|          | 1.0…

To https://huggingface.co/Kurokabe/whisper-large-v2-bn
   4863f2b..ecf5056  main -> main

To https://huggingface.co/Kurokabe/whisper-large-v2-bn
   ecf5056..8e007b1  main -> main



'https://huggingface.co/Kurokabe/whisper-large-v2-bn/commit/ecf5056e09ad06d508761c129fc3127598406e6f'